In [1]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
import glob
from google.cloud import bigquery
import re
from datetime import datetime
import locale




Setor Meio Ambiente em 2022: 19.641. Em 2023, 10623
    Dos 9018 vinculos que foram perdidos em 2023, onde eles estão 2023? 

Setor Desenv. Social e D. Humanos. Em 2022, 83. Em 2023, 1674. ##Foi adicionado os povos INDIGENAS, por isso a subida.
    Dos 1591 vinculos que foram adicionados em 2023, onde eles estavam em 2022?

```



In [2]:
query = """ 
SELECT ANO as ano, MES as mes, CPF as cpf, NOME as nome, Id_SERVIDOR_PORTAL as id_servidor, TIPO_APOSENTADORIA as tipo_aposentadoria, 
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO 
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total, DATA_APOSENTADORIA, DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO, DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_nov_siape_aposentados_cadastro` 
            UNION ALL
SELECT ANO as ano, MES as mes, CPF as cpf, NOME as nome, Id_SERVIDOR_PORTAL as id_servidor, TIPO_APOSENTADORIA as tipo_aposentadoria, 
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO 
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total, DATA_APOSENTADORIA, DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO, DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2021_nov_siape_aposentados_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, CPF as cpf, NOME as nome, Id_SERVIDOR_PORTAL as id_servidor, TIPO_APOSENTADORIA as tipo_aposentadoria, 
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO 
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total, DATA_APOSENTADORIA, DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO, DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2022_nov_siape_aposentados_cadastro` 
            UNION ALL
                SELECT ANO as ano, MES as mes, CPF as cpf, NOME as nome, Id_SERVIDOR_PORTAL as id_servidor, TIPO_APOSENTADORIA as tipo_aposentadoria, 
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO 
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total, DATA_APOSENTADORIA, DESCRICAO_CARGO, DATA_INGRESSO_CARGOFUNCAO, DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2023_nov_siape_aposentados_cadastro`
            """

In [197]:
query1 = """
        SELECT * FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2023_mar_siape_aposentados_cadastro` 
        
        """

In [3]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO
0,2020,11,***.019.452-**,JOAO CARVALHO DA SILVA,82728133,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,29/06/2004,CABO,29/07/2016,03/02/1986
1,2020,11,***.637.642-**,JORGE CUNHA NUNES,78812265,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,15/05/2002,CABO,29/07/2016,01/06/1984
2,2020,11,***.103.432-**,PEDRO VANALDO SAMPAIO BONFIM,84018138,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,15/09/1984,CABO,29/07/2016,01/03/1974
3,2020,11,***.164.202-**,ZENITO GEMAQUE CAVALHEIRO,75819812,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,17/09/1997,CABO,29/07/2016,03/02/1986
4,2020,11,***.907.703-**,ALCIDES SILVA BARBOSA,78819554,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,01/07/2004,CABO,05/03/1980,05/03/1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689358,2022,11,***.656.162-**,ZULMIRA COSTA DOS SANTOS,85315954,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,23/09/2005,PROFESSOR DO ENSINO BASICO EX-TERRITORIO,01/03/2013,27/02/1980
1689359,2022,11,***.586.448-**,ARMANDO ISMAIL OLLAIK,75423443,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,01/04/1991,TEC DE PLANEJ E PESQUISA-QUADRO SUPLEMEN,16/07/2015,01/03/1990
1689360,2022,11,***.774.001-**,MARIA CECILIA COSTA PEREZ,85017445,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,01/10/2021,TEC DE PLANEJ E PESQUISA-QUADRO SUPLEMEN,01/09/1986,01/09/1986
1689361,2022,11,***.526.237-**,ALESSANDRO PORTO SILVA,76721268,APOSENTADORIA POR INVALIDEZ,Ministério da Defesa,REGIME JURIDICO UNICO,1,06/12/2017,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,28/11/2006,22/11/2006


### Ministério da Agricultura, pecuária e abastecimentos -  Categoria M.A., Agricultura e Agropecuária

In [5]:
df

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO
0,2024,3,***.568.267-**,MARIA PAULA CORREIA LEAL,3732496,APOSENTADORIA POR INVALIDEZ,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,30/04/2008,ATOR,01/07/2006,12/08/1982
1,2024,3,***.599.399-**,ALBENER AMOS DE MOURA,1252703,APOSENTADORIA VOLUNTARIA,MINISTERIO DA CULTURA,REGIME JURIDICO UNICO,1,07/04/2017,ATOR,01/11/2004,01/08/1981
2,2024,3,***.565.532-**,ADAILTON CARMONA PARANHOS DOS SANTOS,2357916,APOSENTADORIA VOLUNTARIA,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,12/04/2002,CABO,29/07/2016,15/03/1990
3,2024,3,***.346.032-**,AGNALDO BALIEIRO DA GAMA,1820808,APOSENTADORIA VOLUNTARIA,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,12/05/2011,CABO,29/07/2016,01/10/1990
4,2024,3,***.983.262-**,ANTONIO MENDES PANTOJA,3231250,APOSENTADORIA VOLUNTARIA,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,14/06/2002,CABO,29/07/2016,15/02/1977
...,...,...,...,...,...,...,...,...,...,...,...,...,...
837170,2023,3,***.909.261-**,JOSE CHAVES FILHO,86314779,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,30/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/09/1979,01/09/1979
837171,2023,3,***.245.051-**,JOSE VALDERI PASSOS,75606785,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,02/04/2012,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,17/09/1980,17/09/1980
837172,2023,3,***.319.427-**,RAIMUNDO XIMENES BRAGA,83921997,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,06/09/1994,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/07/2002,01/04/1984
837173,2023,3,***.688.021-**,WALDEMAR OLIVEIRA DE ANDRADE FILHO,81714532,APOSENTADORIA POR INVALIDEZ,Ministério da Defesa,REGIME JURIDICO UNICO,1,15/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,15/09/2009,01/09/1977


In [36]:
df24= df[df['ano']==2024] 

In [37]:
df24['org_lotacao'].unique()

array(['MIN GESTAO E INOV EM SERV PUBLICOS', 'MINISTERIO DA CULTURA',
       'Ministério da Saúde', 'Ministério da Defesa',
       'Ministério da Educação', 'MINISTERIO DOS POVOS INDIGENAS',
       'Presidência da República', 'Ministério de Minas e Energia',
       'Ministério da Justiça e Segurança Pública',
       'Ministério da Ciência, Tecnologia, Inovações e Comunicações',
       'MINISTERIO DOS TRANSPORTES', 'MINISTERIO DA PREVIDENCIA SOCIAL',
       'MINISTERIO DO PLANEJAMENTO E ORCAMENTO', 'MINISTERIO DA FAZENDA',
       'Ministério das Relações Exteriores',
       'MIN DA INTEG E DO DESENV REGIONAL',
       'MIN DO DESENV AGR E AGRIC FAMILIAR',
       'MINISTERIO DO MEIO AMBIENTE', 'Ministério do Turismo',
       'MINISTERIO DO TRABALHO E EMPREGO',
       'MINISTERIO DE PORTOS E AEROPORTOS',
       'MIN DESENVOLV IND COMERCIO E SERVICOS',
       'Controladoria-Geral da União', 'MINISTERIO DAS COMUNICACOES',
       'Ministério da Agricultura, Pecuária e Abastecimento',
       '

In [38]:
df23 = df[df['ano']==2023]
df23

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO
152719,2023,3,***.568.267-**,MARIA PAULA CORREIA LEAL,73827794,APOSENTADORIA POR INVALIDEZ,Ministério da Economia,REGIME JURIDICO UNICO,1,30/04/2008,ATOR,01/07/2006,12/08/1982
152720,2023,3,***.599.399-**,ALBENER AMOS DE MOURA,84517192,APOSENTADORIA VOLUNTARIA,Ministério do Turismo,REGIME JURIDICO UNICO,1,07/04/2017,ATOR,01/11/2004,01/08/1981
152721,2023,3,***.565.532-**,ADAILTON CARMONA PARANHOS DOS SANTOS,72705808,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/04/2002,CABO,29/07/2016,15/03/1990
152722,2023,3,***.346.032-**,AGNALDO BALIEIRO DA GAMA,75019254,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/05/2011,CABO,29/07/2016,01/10/1990
152723,2023,3,***.983.262-**,ANTONIO MENDES PANTOJA,85509122,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,14/06/2002,CABO,29/07/2016,15/02/1977
...,...,...,...,...,...,...,...,...,...,...,...,...,...
837170,2023,3,***.909.261-**,JOSE CHAVES FILHO,86314779,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,30/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/09/1979,01/09/1979
837171,2023,3,***.245.051-**,JOSE VALDERI PASSOS,75606785,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,02/04/2012,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,17/09/1980,17/09/1980
837172,2023,3,***.319.427-**,RAIMUNDO XIMENES BRAGA,83921997,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,06/09/1994,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/07/2002,01/04/1984
837173,2023,3,***.688.021-**,WALDEMAR OLIVEIRA DE ANDRADE FILHO,81714532,APOSENTADORIA POR INVALIDEZ,Ministério da Defesa,REGIME JURIDICO UNICO,1,15/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,15/09/2009,01/09/1977


In [39]:
df23['org_lotacao'].unique() #tem agricultura, pec e abastecimento 

array(['Ministério da Economia', 'Ministério do Turismo',
       'Ministério da Saúde', 'Ministério da Defesa',
       'Ministério da Educação',
       'Ministério da Justiça e Segurança Pública',
       'Presidência da República', 'Ministério de Minas e Energia',
       'Ministério da Ciência, Tecnologia, Inovações e Comunicações',
       'Ministério da Infraestrutura',
       'Ministério das Relações Exteriores',
       'Ministério da Agricultura, Pecuária e Abastecimento',
       'Ministério do Desenvolvimento Regional',
       'Ministério do Meio Ambiente', 'Controladoria-Geral da União',
       'Ministério do Desenvolvimento Social',
       'MINISTERIO DAS COMUNICACOES', 'Defensoria Pública da União'],
      dtype=object)

In [42]:
df23['cpf_nome'] = df23['cpf'] + df23['nome'] 


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_17080\1679446802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df23['cpf_nome'] = df23['cpf'] + df23['nome']


In [44]:
df24['cpf_nome'] = df24['cpf'] + df24['nome']
df24

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_17080\3330751872.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df24['cpf_nome'] = df24['cpf'] + df24['nome']


,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome
0,2024,3,***.568.267-**,MARIA PAULA CORREIA LEAL,3732496,APOSENTADORIA POR INVALIDEZ,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,30/04/2008,ATOR,01/07/2006,12/08/1982,***.568.267-**MARIA PAULA CORREIA LEAL
1,2024,3,***.599.399-**,ALBENER AMOS DE MOURA,1252703,APOSENTADORIA VOLUNTARIA,MINISTERIO DA CULTURA,REGIME JURIDICO UNICO,1,07/04/2017,ATOR,01/11/2004,01/08/1981,***.599.399-**ALBENER AMOS DE MOURA
2,2024,3,***.565.532-**,ADAILTON CARMONA PARANHOS DOS SANTOS,2357916,APOSENTADORIA VOLUNTARIA,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,12/04/2002,CABO,29/07/2016,15/03/1990,***.565.532-**ADAILTON CARMONA PARANHOS DOS SA...
3,2024,3,***.346.032-**,AGNALDO BALIEIRO DA GAMA,1820808,APOSENTADORIA VOLUNTARIA,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,12/05/2011,CABO,29/07/2016,01/10/1990,***.346.032-**AGNALDO BALIEIRO DA GAMA
4,2024,3,***.983.262-**,ANTONIO MENDES PANTOJA,3231250,APOSENTADORIA VOLUNTARIA,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,14/06/2002,CABO,29/07/2016,15/02/1977,***.983.262-**ANTONIO MENDES PANTOJA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718996,2024,3,***.909.261-**,JOSE CHAVES FILHO,3253532,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,30/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/09/1979,01/09/1979,***.909.261-**JOSE CHAVES FILHO
718997,2024,3,***.245.051-**,JOSE VALDERI PASSOS,1841836,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,02/04/2012,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,17/09/1980,17/09/1980,***.245.051-**JOSE VALDERI PASSOS
718998,2024,3,***.319.427-**,RAIMUNDO XIMENES BRAGA,2751874,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,06/09/1994,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/07/2002,01/04/1984,***.319.427-**RAIMUNDO XIMENES BRAGA
718999,2024,3,***.688.021-**,WALDEMAR OLIVEIRA DE ANDRADE FILHO,118493,APOSENTADORIA POR INVALIDEZ,Ministério da Defesa,REGIME JURIDICO UNICO,1,15/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,15/09/2009,01/09/1977,***.688.021-**WALDEMAR OLIVEIRA DE ANDRADE FILHO


In [46]:
cpfsnomes = df24[df24['cpf_nome'].isin(df23['cpf_nome'])] 
cpfsnomes

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome
0,2024,3,***.568.267-**,MARIA PAULA CORREIA LEAL,3732496,APOSENTADORIA POR INVALIDEZ,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,30/04/2008,ATOR,01/07/2006,12/08/1982,***.568.267-**MARIA PAULA CORREIA LEAL
1,2024,3,***.599.399-**,ALBENER AMOS DE MOURA,1252703,APOSENTADORIA VOLUNTARIA,MINISTERIO DA CULTURA,REGIME JURIDICO UNICO,1,07/04/2017,ATOR,01/11/2004,01/08/1981,***.599.399-**ALBENER AMOS DE MOURA
2,2024,3,***.565.532-**,ADAILTON CARMONA PARANHOS DOS SANTOS,2357916,APOSENTADORIA VOLUNTARIA,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,12/04/2002,CABO,29/07/2016,15/03/1990,***.565.532-**ADAILTON CARMONA PARANHOS DOS SA...
3,2024,3,***.346.032-**,AGNALDO BALIEIRO DA GAMA,1820808,APOSENTADORIA VOLUNTARIA,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,12/05/2011,CABO,29/07/2016,01/10/1990,***.346.032-**AGNALDO BALIEIRO DA GAMA
4,2024,3,***.983.262-**,ANTONIO MENDES PANTOJA,3231250,APOSENTADORIA VOLUNTARIA,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,14/06/2002,CABO,29/07/2016,15/02/1977,***.983.262-**ANTONIO MENDES PANTOJA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718996,2024,3,***.909.261-**,JOSE CHAVES FILHO,3253532,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,30/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/09/1979,01/09/1979,***.909.261-**JOSE CHAVES FILHO
718997,2024,3,***.245.051-**,JOSE VALDERI PASSOS,1841836,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,02/04/2012,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,17/09/1980,17/09/1980,***.245.051-**JOSE VALDERI PASSOS
718998,2024,3,***.319.427-**,RAIMUNDO XIMENES BRAGA,2751874,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,06/09/1994,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/07/2002,01/04/1984,***.319.427-**RAIMUNDO XIMENES BRAGA
718999,2024,3,***.688.021-**,WALDEMAR OLIVEIRA DE ANDRADE FILHO,118493,APOSENTADORIA POR INVALIDEZ,Ministério da Defesa,REGIME JURIDICO UNICO,1,15/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,15/09/2009,01/09/1977,***.688.021-**WALDEMAR OLIVEIRA DE ANDRADE FILHO


In [48]:
cpfsnomes['ano'].unique()

<IntegerArray>
[2024]
Length: 1, dtype: Int64

In [47]:
cpfsnomes['org_lotacao'].value_counts()

Ministério da Educação                                         114654
Ministério da Saúde                                             94563
MIN GESTAO E INOV EM SERV PUBLICOS                              64985
MINISTERIO DA PREVIDENCIA SOCIAL                                37533
Ministério da Defesa                                            24224
Ministério da Justiça e Segurança Pública                       14780
Presidência da República                                        13510
Ministério da Ciência, Tecnologia, Inovações e Comunicações      8428
MINISTERIO DO PLANEJAMENTO E ORCAMENTO                           8043
MIN DO DESENV AGR E AGRIC FAMILIAR                               4974
MINISTERIO DO MEIO AMBIENTE                                      4361
MIN DA INTEG E DO DESENV REGIONAL                                3150
MINISTERIO DA CULTURA                                            1973
MINISTERIO DOS POVOS INDIGENAS                                   1622
Ministério das Relaç

In [49]:
cpfsnomes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404068 entries, 0 to 719000
Data columns (total 14 columns):
 #   Column                                Non-Null Count   Dtype 
---  ------                                --------------   ----- 
 0   ano                                   404068 non-null  Int64 
 1   mes                                   404068 non-null  Int64 
 2   cpf                                   404068 non-null  object
 3   nome                                  404068 non-null  object
 4   id_servidor                           404068 non-null  Int64 
 5   tipo_aposentadoria                    404068 non-null  object
 6   org_lotacao                           404068 non-null  object
 7   regime_juridico                       404068 non-null  object
 8   total                                 404068 non-null  Int64 
 9   DATA_APOSENTADORIA                    403288 non-null  object
 10  DESCRICAO_CARGO                       404068 non-null  object
 11  DATA_INGRESSO

In [ ]:

duplicatas = list(df23.duplicated(subset=['cpf_nome'], keep=False)) 
duplicatas
df23['duplicado_cpf_nome']=duplicatas

cpfs = df23[df23['cpf'].isin(df24['cpf'])]
cpfs


### Previdência Social

In [28]:
df24= df[(df['ano']==2024) & (df['org_lotacao']=='MINISTERIO DA PREVIDENCIA SOCIAL')]

In [29]:
df23= df[df['ano']==2023]

In [30]:
df23['org_lotacao'].unique() #em 2023 nao tem o órgão da prev. social.

array(['Ministério da Economia', 'Ministério do Turismo',
       'Ministério da Saúde', 'Ministério da Defesa',
       'Ministério da Educação',
       'Ministério da Justiça e Segurança Pública',
       'Presidência da República', 'Ministério de Minas e Energia',
       'Ministério da Ciência, Tecnologia, Inovações e Comunicações',
       'Ministério da Infraestrutura',
       'Ministério das Relações Exteriores',
       'Ministério da Agricultura, Pecuária e Abastecimento',
       'Ministério do Desenvolvimento Regional',
       'Ministério do Meio Ambiente', 'Controladoria-Geral da União',
       'Ministério do Desenvolvimento Social',
       'MINISTERIO DAS COMUNICACOES', 'Defensoria Pública da União'],
      dtype=object)

In [31]:
np.set_printoptions(threshold=9000000)


In [32]:
id_prev_2024 = df24['id_servidor'].unique()
id_prev_2024

<IntegerArray>
[1489505, 1123873, 3629843,  950062, 1349431, 2503358, 1817398, 1922191,
 3194261, 3017586,
 ...
 1370547, 2255252, 1954464,  690489,  927325,  365337,  638106,  610692,
 1734382, 1970999]
Length: 38289, dtype: Int64

In [33]:
cpf_prev_2024 = df24['cpf'].unique()
cpf_prev_2024

array(['***.531.367-**', '***.517.872-**', '***.778.557-**',
       '***.488.214-**', '***.280.794-**', '***.022.884-**',
       '***.216.722-**', '***.053.154-**', '***.281.198-**',
       '***.673.754-**', '***.938.505-**', '***.283.124-**',
       '***.137.408-**', '***.949.493-**', '***.862.715-**',
       '***.295.254-**', '***.587.713-**', '***.909.170-**',
       '***.724.610-**', '***.735.459-**', '***.017.104-**',
       '***.338.900-**', '***.095.088-**', '***.349.755-**',
       '***.373.687-**', '***.964.454-**', '***.771.714-**',
       '***.823.788-**', '***.125.968-**', '***.322.365-**',
       '***.118.397-**', '***.204.983-**', '***.541.487-**',
       '***.775.287-**', '***.050.548-**', '***.257.327-**',
       '***.224.868-**', '***.316.309-**', '***.258.384-**',
       '***.819.747-**', '***.267.801-**', '***.846.005-**',
       '***.138.098-**', '***.947.847-**', '***.529.694-**',
       '***.846.084-**', '***.601.242-**', '***.044.127-**',
       '***.140.547-**',

In [34]:
df23['org_lotacao'].unique()

array(['Ministério da Economia', 'Ministério do Turismo',
       'Ministério da Saúde', 'Ministério da Defesa',
       'Ministério da Educação',
       'Ministério da Justiça e Segurança Pública',
       'Presidência da República', 'Ministério de Minas e Energia',
       'Ministério da Ciência, Tecnologia, Inovações e Comunicações',
       'Ministério da Infraestrutura',
       'Ministério das Relações Exteriores',
       'Ministério da Agricultura, Pecuária e Abastecimento',
       'Ministério do Desenvolvimento Regional',
       'Ministério do Meio Ambiente', 'Controladoria-Geral da União',
       'Ministério do Desenvolvimento Social',
       'MINISTERIO DAS COMUNICACOES', 'Defensoria Pública da União'],
      dtype=object)

In [11]:
df23

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total
69523,2023,3,***.503.982-**,MARILIA DULCE OLIVEIRA DA SILVA SANTOS,76011206,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
69524,2023,3,***.336.122-**,ANTONIO CARLOS PEREIRA,79914308,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
69525,2023,3,***.186.828-**,ANTONIO CARLOS RIBEIRO,74124445,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
69526,2023,3,***.616.128-**,APARECIDO PEREIRA,78925586,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
69527,2023,3,***.340.137-**,DANIEL ROCHA,73425016,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
...,...,...,...,...,...,...,...,...,...
1936103,2023,3,***.174.397-**,LUCIA HELENA SALGADO E SILVA PEDRA,77821861,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
1936104,2023,3,***.877.897-**,RONALDO SEROA DA MOTTA,77219270,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
1936105,2023,3,***.295.551-**,JOAO TAVARES DE ABREU,82123662,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1
1936106,2023,3,***.245.051-**,JOSE VALDERI PASSOS,75606785,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1


In [12]:
df24

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total
35133,2024,3,***.488.214-**,ALEXANDRE MARCOS DA SILVEIRA ARRAES,950062,APOSENTADORIA VOLUNTARIA,MINISTERIO DA PREVIDENCIA SOCIAL,REGIME JURIDICO UNICO,1
35134,2024,3,***.053.154-**,AMOS OLIVEIRA DE ASSIS,1922191,APOSENTADORIA VOLUNTARIA,MINISTERIO DA PREVIDENCIA SOCIAL,REGIME JURIDICO UNICO,1
35135,2024,3,***.283.124-**,ARISTOTELES DE SIQUEIRA CAMPOS ARAUJO,2157112,APOSENTADORIA VOLUNTARIA,MINISTERIO DA PREVIDENCIA SOCIAL,REGIME JURIDICO UNICO,1
35136,2024,3,***.909.170-**,CLAITON DA SILVA SANTOS,3498877,APOSENTADORIA VOLUNTARIA,MINISTERIO DA PREVIDENCIA SOCIAL,REGIME JURIDICO UNICO,1
35137,2024,3,***.095.088-**,EBE SBRIGHI PEREIRA,3455478,APOSENTADORIA VOLUNTARIA,MINISTERIO DA PREVIDENCIA SOCIAL,REGIME JURIDICO UNICO,1
...,...,...,...,...,...,...,...,...,...
1965577,2024,3,***.356.144-**,SERGIO TAVARES PEREIRA,344138,APOSENTADORIA VOLUNTARIA,MINISTERIO DA PREVIDENCIA SOCIAL,REGIME JURIDICO UNICO,1
1965578,2024,3,***.971.006-**,THIAGO VASCONCELOS PIMENTA,40002,APOSENTADORIA POR INVALIDEZ,MINISTERIO DA PREVIDENCIA SOCIAL,REGIME JURIDICO UNICO,1
1965579,2024,3,***.349.364-**,VERA LUCE BERNARDES CARDOSO,2019008,APOSENTADORIA VOLUNTARIA,MINISTERIO DA PREVIDENCIA SOCIAL,REGIME JURIDICO UNICO,1
1965580,2024,3,***.843.894-**,ZELDA DO MONTE ROSSITER,2103062,APOSENTADORIA VOLUNTARIA,MINISTERIO DA PREVIDENCIA SOCIAL,REGIME JURIDICO UNICO,1


In [35]:
df23[df23['nome']=='EBE SBRIGHI PEREIRA']

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO
2313,2023,3,***.095.088-**,EBE SBRIGHI PEREIRA,78424681,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,14/09/1994,MEDICO,31/03/2006,18/09/1967


In [14]:
df24[df24['nome']=='EBE SBRIGHI PEREIRA']

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total
35137,2024,3,***.095.088-**,EBE SBRIGHI PEREIRA,3455478,APOSENTADORIA VOLUNTARIA,MINISTERIO DA PREVIDENCIA SOCIAL,REGIME JURIDICO UNICO,1


In [12]:
intersec_cpfs = df23[df23['cpf'].isin(cpf_prev_2024)]
intersec_cpfs

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total
139863,2023,3,***.633.507-**,FERNANDO MARCHIORI COSTA,85619710,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
139872,2023,3,***.025.544-**,GILMAR RODRIGUES BARBOSA,83815385,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
139941,2023,3,***.562.507-**,JOSE PONTES VIEIRA,73724012,APOSENTADORIA POR INVALIDEZ,Ministério da Economia,REGIME JURIDICO UNICO,1
139966,2023,3,***.768.804-**,VERA LUCIA DE HOLANDA LOPES,84320326,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
139982,2023,3,***.653.677-**,JAIME LOUREIRO NOBRE BAPTISTA,74009667,APOSENTADORIA VOLUNTARIA,Ministério da Saúde,REGIME JURIDICO UNICO,1
...,...,...,...,...,...,...,...,...,...
2113462,2023,3,***.637.622-**,MARIA DAS NEVES SOARES DA SILVA,81614034,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
2113472,2023,3,***.038.982-**,MARIA DE NAZARE PACHECO DE SOUZA,80110836,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
2113504,2023,3,***.212.562-**,MARIA IZABEL ISACKSSON,82506095,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
2113614,2023,3,***.738.802-**,ONEIDE SANTIAGO DE OLIVEIRA,77727658,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1


In [36]:
intersec_cpfs.duplicated()

139863     False
139872     False
139941     False
139966     False
139982     False
           ...  
2113462    False
2113472     True
2113504    False
2113614    False
2113674    False
Length: 55602, dtype: bool

In [37]:
intersec_cpfs['org_lotacao'].value_counts() 
## De 55602 pessoas em 2023, 41130 são do ministério da economia. 
#No entanto, como pode haver tantos sendo que apenas 38298 são do Ministério da Previdência em 2024? => Dados estão duplicados


Ministério da Economia                                         41130
Ministério da Educação                                          5257
Ministério da Saúde                                             5093
Ministério da Defesa                                            1428
Ministério da Justiça e Segurança Pública                        752
Ministério da Agricultura, Pecuária e Abastecimento              574
Ministério da Ciência, Tecnologia, Inovações e Comunicações      479
Ministério do Meio Ambiente                                      200
Presidência da República                                         167
Ministério do Turismo                                            157
Ministério do Desenvolvimento Regional                           114
Ministério da Infraestrutura                                      87
Ministério das Relações Exteriores                                80
Ministério de Minas e Energia                                     47
Controladoria-Geral da União      

In [38]:
df23[df23['id_servidor'].isin(id_prev_2024)] ## Não encontro as mesmas informações porque os IDs mudam de um ano pra outro

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO


In [39]:
df23['cpf'] = df23['cpf'].apply(lambda x: re.sub(r'[^0-9]', '', x)) #tirando os caracteres especiais da coluna 'cpf'

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_14548\3118172672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df23['cpf'] = df23['cpf'].apply(lambda x: re.sub(r'[^0-9]', '', x)) #tirando os caracteres especiais da coluna 'cpf'


In [40]:
df23['cpf_nome'] = df23['cpf'] + df23['nome']
df23

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_14548\645293855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df23['cpf_nome'] = df23['cpf'] + df23['nome']


,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome
0,2023,3,568267,MARIA PAULA CORREIA LEAL,73827794,APOSENTADORIA POR INVALIDEZ,Ministério da Economia,REGIME JURIDICO UNICO,1,30/04/2008,ATOR,01/07/2006,12/08/1982,568267MARIA PAULA CORREIA LEAL
1,2023,3,599399,ALBENER AMOS DE MOURA,84517192,APOSENTADORIA VOLUNTARIA,Ministério do Turismo,REGIME JURIDICO UNICO,1,07/04/2017,ATOR,01/11/2004,01/08/1981,599399ALBENER AMOS DE MOURA
2,2023,3,565532,ADAILTON CARMONA PARANHOS DOS SANTOS,72705808,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/04/2002,CABO,29/07/2016,15/03/1990,565532ADAILTON CARMONA PARANHOS DOS SANTOS
3,2023,3,346032,AGNALDO BALIEIRO DA GAMA,75019254,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/05/2011,CABO,29/07/2016,01/10/1990,346032AGNALDO BALIEIRO DA GAMA
4,2023,3,983262,ANTONIO MENDES PANTOJA,85509122,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,14/06/2002,CABO,29/07/2016,15/02/1977,983262ANTONIO MENDES PANTOJA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717169,2023,3,909261,JOSE CHAVES FILHO,86314779,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,30/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/09/1979,01/09/1979,909261JOSE CHAVES FILHO
717170,2023,3,245051,JOSE VALDERI PASSOS,75606785,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,02/04/2012,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,17/09/1980,17/09/1980,245051JOSE VALDERI PASSOS
717171,2023,3,319427,RAIMUNDO XIMENES BRAGA,83921997,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,06/09/1994,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/07/2002,01/04/1984,319427RAIMUNDO XIMENES BRAGA
717172,2023,3,688021,WALDEMAR OLIVEIRA DE ANDRADE FILHO,81714532,APOSENTADORIA POR INVALIDEZ,Ministério da Defesa,REGIME JURIDICO UNICO,1,15/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,15/09/2009,01/09/1977,688021WALDEMAR OLIVEIRA DE ANDRADE FILHO


In [41]:

duplicatas = list(df23.duplicated(subset=['cpf_nome'], keep=False)) 
duplicatas

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [42]:
df23['duplicado_cpf_nome']=duplicatas


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_14548\2794277074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df23['duplicado_cpf_nome']=duplicatas


In [43]:
df23[df23['cpf_nome']=='324807MARIA DELZITA NEVES']

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome,duplicado_cpf_nome
841,2023,3,324807,MARIA DELZITA NEVES,73119271,APOSENTADORIA VOLUNTARIA,Ministério da Justiça e Segurança Pública,REGIME JURIDICO UNICO,1,14/01/1992,MEDICO,01/06/2008,24/05/1977,324807MARIA DELZITA NEVES,True
15875,2023,3,324807,MARIA DELZITA NEVES,73119271,APOSENTADORIA VOLUNTARIA,Ministério da Saúde,REGIME JURIDICO UNICO,1,01/09/2010,MEDICO,01/03/2006,10/12/1978,324807MARIA DELZITA NEVES,True


In [50]:
df23

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome,duplicado_cpf_nome
0,2023,3,568267,MARIA PAULA CORREIA LEAL,73827794,APOSENTADORIA POR INVALIDEZ,Ministério da Economia,REGIME JURIDICO UNICO,1,30/04/2008,ATOR,01/07/2006,12/08/1982,568267MARIA PAULA CORREIA LEAL,False
1,2023,3,599399,ALBENER AMOS DE MOURA,84517192,APOSENTADORIA VOLUNTARIA,Ministério do Turismo,REGIME JURIDICO UNICO,1,07/04/2017,ATOR,01/11/2004,01/08/1981,599399ALBENER AMOS DE MOURA,False
2,2023,3,565532,ADAILTON CARMONA PARANHOS DOS SANTOS,72705808,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/04/2002,CABO,29/07/2016,15/03/1990,565532ADAILTON CARMONA PARANHOS DOS SANTOS,False
3,2023,3,346032,AGNALDO BALIEIRO DA GAMA,75019254,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/05/2011,CABO,29/07/2016,01/10/1990,346032AGNALDO BALIEIRO DA GAMA,False
4,2023,3,983262,ANTONIO MENDES PANTOJA,85509122,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,14/06/2002,CABO,29/07/2016,15/02/1977,983262ANTONIO MENDES PANTOJA,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717169,2023,3,909261,JOSE CHAVES FILHO,86314779,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,30/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/09/1979,01/09/1979,909261JOSE CHAVES FILHO,False
717170,2023,3,245051,JOSE VALDERI PASSOS,75606785,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,02/04/2012,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,17/09/1980,17/09/1980,245051JOSE VALDERI PASSOS,False
717171,2023,3,319427,RAIMUNDO XIMENES BRAGA,83921997,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,06/09/1994,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/07/2002,01/04/1984,319427RAIMUNDO XIMENES BRAGA,False
717172,2023,3,688021,WALDEMAR OLIVEIRA DE ANDRADE FILHO,81714532,APOSENTADORIA POR INVALIDEZ,Ministério da Defesa,REGIME JURIDICO UNICO,1,15/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,15/09/2009,01/09/1977,688021WALDEMAR OLIVEIRA DE ANDRADE FILHO,False


In [44]:
## VERIFICAR SE HÁ DUPLICADA DE NOME, CPF E DATA DE NOMEAÇAO PARA O SERVIÇO PÚBLICO

In [46]:
df23_economia = df23[df23['org_lotacao']=='Ministério da Economia']
df23_economia

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome,duplicado_cpf_nome
0,2023,3,568267,MARIA PAULA CORREIA LEAL,73827794,APOSENTADORIA POR INVALIDEZ,Ministério da Economia,REGIME JURIDICO UNICO,1,30/04/2008,ATOR,01/07/2006,12/08/1982,568267MARIA PAULA CORREIA LEAL,False
2,2023,3,565532,ADAILTON CARMONA PARANHOS DOS SANTOS,72705808,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/04/2002,CABO,29/07/2016,15/03/1990,565532ADAILTON CARMONA PARANHOS DOS SANTOS,False
3,2023,3,346032,AGNALDO BALIEIRO DA GAMA,75019254,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/05/2011,CABO,29/07/2016,01/10/1990,346032AGNALDO BALIEIRO DA GAMA,False
4,2023,3,983262,ANTONIO MENDES PANTOJA,85509122,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,14/06/2002,CABO,29/07/2016,15/02/1977,983262ANTONIO MENDES PANTOJA,False
5,2023,3,530592,ANTONIO MENEZES DA SILVA,86308221,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/05/2000,CABO,29/07/2016,04/02/1985,530592ANTONIO MENEZES DA SILVA,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717158,2023,3,877897,RONALDO SEROA DA MOTTA,77219270,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,11/07/2013,TEC DE PLANEJ E PESQUISA-QUADRO SUPLEMEN,30/06/2000,03/05/1979,877897RONALDO SEROA DA MOTTA,False
717159,2023,3,091344,SELMA MARIA GABRIEL CALHEIROS,84224728,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,01/03/1996,TEC DE PLANEJ E PESQUISA-QUADRO SUPLEMEN,25/04/2016,28/02/1972,091344SELMA MARIA GABRIEL CALHEIROS,False
717160,2023,3,891541,SERGIO AUGUSTO BATALHONE,74014098,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,04/01/2021,TEC DE PLANEJ E PESQUISA-QUADRO SUPLEMEN,30/06/2000,10/03/1980,891541SERGIO AUGUSTO BATALHONE,False
717161,2023,3,151203,SILVIO CARACAS DE MOURA JUNIOR,76312522,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,19/09/1996,TEC DE PLANEJ E PESQUISA-QUADRO SUPLEMEN,16/07/2015,27/11/1990,151203SILVIO CARACAS DE MOURA JUNIOR,False


In [47]:
df23_economia['duplicado_cpf_nome'].value_counts()

False    114817
True       1812
Name: duplicado_cpf_nome, dtype: int64

In [48]:
df23_economia[df23_economia['duplicado_cpf_nome']==True] 

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome,duplicado_cpf_nome
652,2023,3,512597,JOSIAS PIRES FERREIRA,83519836,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,None,MAJOR,04/07/2002,24/08/1945,512597JOSIAS PIRES FERREIRA,True
673,2023,3,452107,YVON TOLEDO RODRIGUES,77812443,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,05/07/2002,MAJOR,04/07/2002,13/01/1945,452107YVON TOLEDO RODRIGUES,True
882,2023,3,257734,ABELARDO BARBOSA ESPINDOLA,83406351,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,20/04/1989,MEDICO,01/03/2006,04/12/1952,257734ABELARDO BARBOSA ESPINDOLA,True
883,2023,3,168742,ABELARDO DA SILVA VAZ,81820455,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,05/08/1991,MEDICO,01/07/2006,01/10/1977,168742ABELARDO DA SILVA VAZ,True
890,2023,3,288302,ALBERTO BRITO CALLINS,76920388,APOSENTADORIA POR INVALIDEZ,Ministério da Economia,REGIME JURIDICO UNICO,1,18/07/2012,MEDICO,01/07/2006,04/01/1982,288302ALBERTO BRITO CALLINS,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717018,2023,3,930452,VANDELNIRA MONTEIRO DA COSTA,78520283,APOSENTADORIA POR INVALIDEZ,Ministério da Economia,REGIME JURIDICO UNICO,1,16/09/1997,PROFESSOR DO ENSINO BASICO EX-TERRITORIO,01/03/2013,01/05/1978,930452VANDELNIRA MONTEIRO DA COSTA,True
717031,2023,3,976032,VAUDIMARIAN PEREIRA DE LIMA,79825655,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,26/10/2005,PROFESSOR DO ENSINO BASICO EX-TERRITORIO,01/07/2008,01/07/1979,976032VAUDIMARIAN PEREIRA DE LIMA,True
717033,2023,3,390092,VENUSIA GONCALVES DOS SANTOS,74921933,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,06/06/1990,PROFESSOR DO ENSINO BASICO EX-TERRITORIO,01/03/2013,15/06/1962,390092VENUSIA GONCALVES DOS SANTOS,True
717088,2023,3,553932,YOLANDA PEREIRA SANTOS SILVA,76628092,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,16/06/2011,PROFESSOR DO ENSINO BASICO EX-TERRITORIO,01/03/2013,07/12/1981,553932YOLANDA PEREIRA SANTOS SILVA,True


In [49]:
df23_economia[df23_economia['cpf_nome']=='480082ALICE AUGUSTA MARTINS VENTURA MAGALHAES'] #duplicada

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome,duplicado_cpf_nome
893,2023,3,480082,ALICE AUGUSTA MARTINS VENTURA MAGALHAES,72318779,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,13/03/2007,MEDICO,01/07/2006,01/05/1982,480082ALICE AUGUSTA MARTINS VENTURA MAGALHAES,True
894,2023,3,480082,ALICE AUGUSTA MARTINS VENTURA MAGALHAES,72318779,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,09/10/1993,MEDICO,01/07/2006,06/07/1978,480082ALICE AUGUSTA MARTINS VENTURA MAGALHAES,True


In [78]:
df23['data_diploma_cpf_nome'] = df23['cpf_nome'] + df23['DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO']
df23

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome,data_diploma_cpf_nome,duplicata_data_diploma_cpf_nome
0,2023,3,568267,MARIA PAULA CORREIA LEAL,73827794,APOSENTADORIA POR INVALIDEZ,Ministério da Economia,REGIME JURIDICO UNICO,1,30/04/2008,ATOR,01/07/2006,12/08/1982,568267MARIA PAULA CORREIA LEAL,568267MARIA PAULA CORREIA LEAL12/08/1982,False
1,2023,3,599399,ALBENER AMOS DE MOURA,84517192,APOSENTADORIA VOLUNTARIA,Ministério do Turismo,REGIME JURIDICO UNICO,1,07/04/2017,ATOR,01/11/2004,01/08/1981,599399ALBENER AMOS DE MOURA,599399ALBENER AMOS DE MOURA01/08/1981,False
2,2023,3,565532,ADAILTON CARMONA PARANHOS DOS SANTOS,72705808,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/04/2002,CABO,29/07/2016,15/03/1990,565532ADAILTON CARMONA PARANHOS DOS SANTOS,565532ADAILTON CARMONA PARANHOS DOS SANTOS15/0...,False
3,2023,3,346032,AGNALDO BALIEIRO DA GAMA,75019254,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,12/05/2011,CABO,29/07/2016,01/10/1990,346032AGNALDO BALIEIRO DA GAMA,346032AGNALDO BALIEIRO DA GAMA01/10/1990,False
4,2023,3,983262,ANTONIO MENDES PANTOJA,85509122,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,14/06/2002,CABO,29/07/2016,15/02/1977,983262ANTONIO MENDES PANTOJA,983262ANTONIO MENDES PANTOJA15/02/1977,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717169,2023,3,909261,JOSE CHAVES FILHO,86314779,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,30/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/09/1979,01/09/1979,909261JOSE CHAVES FILHO,909261JOSE CHAVES FILHO01/09/1979,False
717170,2023,3,245051,JOSE VALDERI PASSOS,75606785,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,02/04/2012,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,17/09/1980,17/09/1980,245051JOSE VALDERI PASSOS,245051JOSE VALDERI PASSOS17/09/1980,False
717171,2023,3,319427,RAIMUNDO XIMENES BRAGA,83921997,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,REGIME JURIDICO UNICO,1,06/09/1994,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,01/07/2002,01/04/1984,319427RAIMUNDO XIMENES BRAGA,319427RAIMUNDO XIMENES BRAGA01/04/1984,False
717172,2023,3,688021,WALDEMAR OLIVEIRA DE ANDRADE FILHO,81714532,APOSENTADORIA POR INVALIDEZ,Ministério da Defesa,REGIME JURIDICO UNICO,1,15/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,15/09/2009,01/09/1977,688021WALDEMAR OLIVEIRA DE ANDRADE FILHO,688021WALDEMAR OLIVEIRA DE ANDRADE FILHO01/09/...,False


In [115]:
df1

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,COD_TIPO_APOSENTADORIA,TIPO_APOSENTADORIA,DATA_APOSENTADORIA,DESCRICAO_CARGO,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,ANO,MES,cpf
0,73827794,MARIA PAULA CORREIA LEAL,***.568.267-**,004****,2,APOSENTADORIA POR INVALIDEZ,30/04/2008,ATOR,-3,Inválido,40806,DEP.DE CENTRAL.SERV.DE INATIVOS E PENS.,17000,Ministério da Economia,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/07/2006,NaN,02/01/2019,S/N,12/08/1982,NaN,DECRETO,PORTARIA,2023,3,568267
1,84517192,ALBENER AMOS DE MOURA,***.599.399-**,004****,1,APOSENTADORIA VOLUNTARIA,07/04/2017,ATOR,-3,Inválido,24203,Fundação Nacional de Artes,54000,Ministério do Turismo,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/11/2004,NaN,12/07/2004,S/N,01/08/1981,NaN,PORTARIA,PORTARIA,2023,3,599399
2,72705808,ADAILTON CARMONA PARANHOS DOS SANTOS,***.565.532-**,014****,1,APOSENTADORIA VOLUNTARIA,12/04/2002,CABO,40801000000164,POLICIA MILITAR /EX-TERRITORIO DO AMAPA,40801,Governo do Ex-Território Federal do Amapá,17000,Ministério da Economia,5,Aposentadoria,REFORMA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,29/07/2016,NaN,15/03/1990,NI,15/03/1990,NaN,PORTARIA,PORTARIA,2023,3,565532
3,75019254,AGNALDO BALIEIRO DA GAMA,***.346.032-**,014****,1,APOSENTADORIA VOLUNTARIA,12/05/2011,CABO,40801000000164,POLICIA MILITAR /EX-TERRITORIO DO AMAPA,40801,Governo do Ex-Território Federal do Amapá,17000,Ministério da Economia,5,Aposentadoria,RESERVA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,29/07/2016,NaN,01/10/1990,NI,01/10/1990,NaN,DECRETO,DECRETO,2023,3,346032
4,85509122,ANTONIO MENDES PANTOJA,***.983.262-**,014****,1,APOSENTADORIA VOLUNTARIA,14/06/2002,CABO,40801000000164,POLICIA MILITAR /EX-TERRITORIO DO AMAPA,40801,Governo do Ex-Território Federal do Amapá,17000,Ministério da Economia,5,Aposentadoria,REFORMA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,29/07/2016,NaN,15/02/1977,NI,15/02/1977,NaN,PORTARIA,PORTARIA,2023,3,983262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418787,86314779,JOSE CHAVES FILHO,***.909.261-**,001****,1,APOSENTADORIA VOLUNTARIA,30/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,-3,Inválido,21000,Comando da Aeronáutica,40105,Ministério da Defesa,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/09/1979,NaN,01/09/1979,0120CONF-,01/09/1979,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,909261
418788,75606785,JOSE VALDERI PASSOS,***.245.051-**,001****,1,APOSENTADORIA VOLUNTARIA,02/04/2012,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,-3,Inválido,21000,Comando da Aeronáutica,40105,Ministério da Defesa,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,17/09/1980,NaN,17/09/1980,006 CONF,17/09/1980,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,245051
418789,83921997,RAIMUNDO XIMENES BRAGA,***.319.427-**,001****,1,APOSENTADORIA VOLUNTARIA,06/09/1994,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,-3,Inválido,21000,Comando da Aeronáutica,40105,Ministério da Defesa,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/07/2002,NaN,01/04/1984,01,01/04/1984,NaN,INSTRUCAO NORMATIVA,INSTRUCAO NORMATIVA,2023,3,319427
418790,81714532,WALDEMAR OLIVEIRA DE ANDRADE FILHO,***.688.021-**,001****,2,APOSENTADORIA POR INVALIDEZ,15/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,-3,Inválido,21000,Comando da Aeronáutica,40105,Ministério da Defesa,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,15/09/2009,NaN,01/09/1977,002 CONF,01/09/1977,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,688021


In [199]:
df1['cpf'] = df1['CPF'].apply(lambda x: re.sub(r'[^0-9]', '', x)) #tirando os caracteres especiais da coluna 'cpf'
df1['cpf_nome'] = df1['cpf'] + df1['NOME']

In [200]:
df1['data_diploma_cpf_nome']=df1['cpf_nome'] + df1['DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO']

In [201]:
df1['data_diploma_cpf_nome_aposenta']=df1['cpf_nome'] + df1['DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO']+df1['DATA_APOSENTADORIA']+df1['DESCRICAO_CARGO']

In [202]:
df1

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,COD_TIPO_APOSENTADORIA,TIPO_APOSENTADORIA,DATA_APOSENTADORIA,DESCRICAO_CARGO,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,ANO,MES,cpf,cpf_nome,data_diploma_cpf_nome,data_diploma_cpf_nome_aposenta
0,73827794,MARIA PAULA CORREIA LEAL,***.568.267-**,004****,2,APOSENTADORIA POR INVALIDEZ,30/04/2008,ATOR,-3,Inválido,40806,DEP.DE CENTRAL.SERV.DE INATIVOS E PENS.,17000,Ministério da Economia,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/07/2006,NaN,02/01/2019,S/N,12/08/1982,NaN,DECRETO,PORTARIA,2023,3,568267,568267MARIA PAULA CORREIA LEAL,568267MARIA PAULA CORREIA LEAL12/08/1982,568267MARIA PAULA CORREIA LEAL12/08/198230/04/...
1,84517192,ALBENER AMOS DE MOURA,***.599.399-**,004****,1,APOSENTADORIA VOLUNTARIA,07/04/2017,ATOR,-3,Inválido,24203,Fundação Nacional de Artes,54000,Ministério do Turismo,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/11/2004,NaN,12/07/2004,S/N,01/08/1981,NaN,PORTARIA,PORTARIA,2023,3,599399,599399ALBENER AMOS DE MOURA,599399ALBENER AMOS DE MOURA01/08/1981,599399ALBENER AMOS DE MOURA01/08/198107/04/201...
2,72705808,ADAILTON CARMONA PARANHOS DOS SANTOS,***.565.532-**,014****,1,APOSENTADORIA VOLUNTARIA,12/04/2002,CABO,40801000000164,POLICIA MILITAR /EX-TERRITORIO DO AMAPA,40801,Governo do Ex-Território Federal do Amapá,17000,Ministério da Economia,5,Aposentadoria,REFORMA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,29/07/2016,NaN,15/03/1990,NI,15/03/1990,NaN,PORTARIA,PORTARIA,2023,3,565532,565532ADAILTON CARMONA PARANHOS DOS SANTOS,565532ADAILTON CARMONA PARANHOS DOS SANTOS15/0...,565532ADAILTON CARMONA PARANHOS DOS SANTOS15/0...
3,75019254,AGNALDO BALIEIRO DA GAMA,***.346.032-**,014****,1,APOSENTADORIA VOLUNTARIA,12/05/2011,CABO,40801000000164,POLICIA MILITAR /EX-TERRITORIO DO AMAPA,40801,Governo do Ex-Território Federal do Amapá,17000,Ministério da Economia,5,Aposentadoria,RESERVA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,29/07/2016,NaN,01/10/1990,NI,01/10/1990,NaN,DECRETO,DECRETO,2023,3,346032,346032AGNALDO BALIEIRO DA GAMA,346032AGNALDO BALIEIRO DA GAMA01/10/1990,346032AGNALDO BALIEIRO DA GAMA01/10/199012/05/...
4,85509122,ANTONIO MENDES PANTOJA,***.983.262-**,014****,1,APOSENTADORIA VOLUNTARIA,14/06/2002,CABO,40801000000164,POLICIA MILITAR /EX-TERRITORIO DO AMAPA,40801,Governo do Ex-Território Federal do Amapá,17000,Ministério da Economia,5,Aposentadoria,REFORMA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,29/07/2016,NaN,15/02/1977,NI,15/02/1977,NaN,PORTARIA,PORTARIA,2023,3,983262,983262ANTONIO MENDES PANTOJA,983262ANTONIO MENDES PANTOJA15/02/1977,983262ANTONIO MENDES PANTOJA15/02/197714/06/20...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418787,86314779,JOSE CHAVES FILHO,***.909.261-**,001****,1,APOSENTADORIA VOLUNTARIA,30/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,-3,Inválido,21000,Comando da Aeronáutica,40105,Ministério da Defesa,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/09/1979,NaN,01/09/1979,0120CONF-,01/09/1979,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,909261,909261JOSE CHAVES FILHO,909261JOSE CHAVES FILHO01/09/1979,909261JOSE CHAVES FILHO01/09/197930/09/2009TEC...
418788,75606785,JOSE VALDERI PASSOS,***.245.051-**,001****,1,APOSENTADORIA VOLUNTARIA,02/04/2012,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,-3,Inválido,21000,Comando da Aeronáutica,40105,Ministério da Defesa,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,17/09/1980,NaN,17/09/1980,006 CONF,17/09/1980,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,245051,245051JOSE VALDERI 

In [203]:
duplicatas = list(df1.duplicated(subset=['data_diploma_cpf_nome_aposenta'], keep=False)) 


In [206]:
df1['flag_id']=duplicatas

In [178]:
df23['flag_id'].value_counts()


False    410982
True       7810
Name: flag_id, dtype: int64

In [207]:
df1

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,COD_TIPO_APOSENTADORIA,TIPO_APOSENTADORIA,DATA_APOSENTADORIA,DESCRICAO_CARGO,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,ANO,MES,cpf,cpf_nome,data_diploma_cpf_nome,data_diploma_cpf_nome_aposenta,flag_id
0,73827794,MARIA PAULA CORREIA LEAL,***.568.267-**,004****,2,APOSENTADORIA POR INVALIDEZ,30/04/2008,ATOR,-3,Inválido,40806,DEP.DE CENTRAL.SERV.DE INATIVOS E PENS.,17000,Ministério da Economia,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/07/2006,NaN,02/01/2019,S/N,12/08/1982,NaN,DECRETO,PORTARIA,2023,3,568267,568267MARIA PAULA CORREIA LEAL,568267MARIA PAULA CORREIA LEAL12/08/1982,568267MARIA PAULA CORREIA LEAL12/08/198230/04/...,False
1,84517192,ALBENER AMOS DE MOURA,***.599.399-**,004****,1,APOSENTADORIA VOLUNTARIA,07/04/2017,ATOR,-3,Inválido,24203,Fundação Nacional de Artes,54000,Ministério do Turismo,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/11/2004,NaN,12/07/2004,S/N,01/08/1981,NaN,PORTARIA,PORTARIA,2023,3,599399,599399ALBENER AMOS DE MOURA,599399ALBENER AMOS DE MOURA01/08/1981,599399ALBENER AMOS DE MOURA01/08/198107/04/201...,False
2,72705808,ADAILTON CARMONA PARANHOS DOS SANTOS,***.565.532-**,014****,1,APOSENTADORIA VOLUNTARIA,12/04/2002,CABO,40801000000164,POLICIA MILITAR /EX-TERRITORIO DO AMAPA,40801,Governo do Ex-Território Federal do Amapá,17000,Ministério da Economia,5,Aposentadoria,REFORMA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,29/07/2016,NaN,15/03/1990,NI,15/03/1990,NaN,PORTARIA,PORTARIA,2023,3,565532,565532ADAILTON CARMONA PARANHOS DOS SANTOS,565532ADAILTON CARMONA PARANHOS DOS SANTOS15/0...,565532ADAILTON CARMONA PARANHOS DOS SANTOS15/0...,False
3,75019254,AGNALDO BALIEIRO DA GAMA,***.346.032-**,014****,1,APOSENTADORIA VOLUNTARIA,12/05/2011,CABO,40801000000164,POLICIA MILITAR /EX-TERRITORIO DO AMAPA,40801,Governo do Ex-Território Federal do Amapá,17000,Ministério da Economia,5,Aposentadoria,RESERVA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,29/07/2016,NaN,01/10/1990,NI,01/10/1990,NaN,DECRETO,DECRETO,2023,3,346032,346032AGNALDO BALIEIRO DA GAMA,346032AGNALDO BALIEIRO DA GAMA01/10/1990,346032AGNALDO BALIEIRO DA GAMA01/10/199012/05/...,False
4,85509122,ANTONIO MENDES PANTOJA,***.983.262-**,014****,1,APOSENTADORIA VOLUNTARIA,14/06/2002,CABO,40801000000164,POLICIA MILITAR /EX-TERRITORIO DO AMAPA,40801,Governo do Ex-Território Federal do Amapá,17000,Ministério da Economia,5,Aposentadoria,REFORMA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,29/07/2016,NaN,15/02/1977,NI,15/02/1977,NaN,PORTARIA,PORTARIA,2023,3,983262,983262ANTONIO MENDES PANTOJA,983262ANTONIO MENDES PANTOJA15/02/1977,983262ANTONIO MENDES PANTOJA15/02/197714/06/20...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418787,86314779,JOSE CHAVES FILHO,***.909.261-**,001****,1,APOSENTADORIA VOLUNTARIA,30/09/2009,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,-3,Inválido,21000,Comando da Aeronáutica,40105,Ministério da Defesa,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/09/1979,NaN,01/09/1979,0120CONF-,01/09/1979,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,909261,909261JOSE CHAVES FILHO,909261JOSE CHAVES FILHO01/09/1979,909261JOSE CHAVES FILHO01/09/197930/09/2009TEC...,False
418788,75606785,JOSE VALDERI PASSOS,***.245.051-**,001****,1,APOSENTADORIA VOLUNTARIA,02/04/2012,TEC DE PROG OP DE DEF AEREA CONT TRAFEGO,-3,Inválido,21000,Comando da Aeronáutica,40105,Ministério da Defesa,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,17/09/1980,NaN,17/09/1980,006 CONF,17/09/1980,NaN,BOLETIM INTERNO,B

In [209]:
df1[df1['NOME']=="PEDRO COSTA SILVA"]

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,COD_TIPO_APOSENTADORIA,TIPO_APOSENTADORIA,DATA_APOSENTADORIA,DESCRICAO_CARGO,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,ANO,MES,cpf,cpf_nome,data_diploma_cpf_nome,data_diploma_cpf_nome_aposenta,flag_id
84149,160700044,PEDRO COSTA SILVA,***.774.512-**,032****,1,APOSENTADORIA VOLUNTARIA,None,SOLDADO 1 CLASSE,40801000000164,POLICIA MILITAR /EX-TERRITORIO DO AMAPA,40801,Governo do Ex-Território Federal do Amapá,17000,Ministério da Economia,5,Aposentadoria,RESERVA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,19/06/2020,NaN,08/03/2021,00311,19/06/2020,NaN,PARECER,PARECER,2023,3,774512,774512PEDRO COSTA SILVA,774512PEDRO COSTA SILVA19/06/2020,NaN,True


In [188]:
df23[df23['nome']=="PEDRO COSTA SILVA"]

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome,data_diploma_cpf_nome,duplicata_data_diploma_cpf_nome,data_diploma_cpf_nome_aposenta,flag_id
84149,2023,3,774512,PEDRO COSTA SILVA,160700044,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1,None,SOLDADO 1 CLASSE,19/06/2020,19/06/2020,774512PEDRO COSTA SILVA,774512PEDRO COSTA SILVA19/06/2020,False,NaN,True


In [221]:
df1[(df1['flag_id']==True) & (df1['DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO'].notnull()) & (df1['DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO'].notnull())]

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,COD_TIPO_APOSENTADORIA,TIPO_APOSENTADORIA,DATA_APOSENTADORIA,DESCRICAO_CARGO,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,ANO,MES,cpf,cpf_nome,data_diploma_cpf_nome,data_diploma_cpf_nome_aposenta,flag_id
31,74124445,ANTONIO CARLOS RIBEIRO,***.186.828-**,021****,1,APOSENTADORIA VOLUNTARIA,None,CABO,40803000000341,POLICIA MILITAR / RO - EC/38,40803,Governo do Ex-Território Federal de Rondônia,17000,Ministério da Economia,5,Aposentadoria,RESERVA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,04/06/2014,NaN,04/06/2014,251/2014,04/06/2014,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,186828,186828ANTONIO CARLOS RIBEIRO,186828ANTONIO CARLOS RIBEIRO04/06/2014,NaN,True
32,72212058,ANTONIO RODRIGUES DOS SANTOS,***.447.062-**,021****,1,APOSENTADORIA VOLUNTARIA,None,CABO,40803000000341,POLICIA MILITAR / RO - EC/38,40803,Governo do Ex-Território Federal de Rondônia,17000,Ministério da Economia,5,Aposentadoria,REFORMA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,04/06/2014,NaN,04/06/2014,251/2014,04/06/2014,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,447062,447062ANTONIO RODRIGUES DOS SANTOS,447062ANTONIO RODRIGUES DOS SANTOS04/06/2014,NaN,True
36,77906826,CLOVIS RIBEIRO DE BRAGA,***.552.449-**,020****,1,APOSENTADORIA VOLUNTARIA,None,CABO,40803000000341,POLICIA MILITAR / RO - EC/38,40803,Governo do Ex-Território Federal de Rondônia,17000,Ministério da Economia,5,Aposentadoria,REFORMA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,01/01/2013,NaN,01/01/2013,2788/20,01/01/2013,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,552449,552449CLOVIS RIBEIRO DE BRAGA,552449CLOVIS RIBEIRO DE BRAGA01/01/2013,NaN,True
40,74505599,ELIEL PEREIRA,***.785.802-**,021****,1,APOSENTADORIA VOLUNTARIA,None,CABO,40803000000341,POLICIA MILITAR / RO - EC/38,40803,Governo do Ex-Território Federal de Rondônia,17000,Ministério da Economia,5,Aposentadoria,RESERVA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,04/06/2014,NaN,04/06/2014,251/2014,04/06/2014,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,785802,785802ELIEL PEREIRA,785802ELIEL PEREIRA04/06/2014,NaN,True
42,72609339,FRANCISCO DE ASSIS DE ANDRADE,***.640.458-**,021****,1,APOSENTADORIA VOLUNTARIA,None,CABO,40803000000341,POLICIA MILITAR / RO - EC/38,40803,Governo do Ex-Território Federal de Rondônia,17000,Ministério da Economia,5,Aposentadoria,RESERVA CBM / PM,REGIME MILITAR,40 HORAS SEMANAIS,04/06/2014,NaN,04/06/2014,251/2014,04/06/2014,NaN,BOLETIM INTERNO,BOLETIM INTERNO,2023,3,640458,640458FRANCISCO DE ASSIS DE ANDRADE,640458FRANCISCO DE ASSIS DE ANDRADE04/06/2014,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399558,81205906,MARIA DE NAZARE DANTAS FREITAS,***.092.012-**,013****,1,APOSENTADORIA VOLUNTARIA,01/12/2017,AUXILIAR OPERACIONAL SERV DIVERSOS - NA,-3,Inválido,40806,DEP.DE CENTRAL.SERV.DE INATIVOS E PENS.,17000,Ministério da Economia,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/09/1983,NaN,05/02/2021,NI,01/09/1983,NaN,DECRETO,CONTRATO,2023,3,092012,092012MARIA DE NAZARE DANTAS FREITAS,092012MARIA DE NAZARE DANTAS FREITAS01/09/1983,092012MARIA DE NAZARE DANTAS FREITAS01/09/1983...,True
408887,80914766,MARCELO RAMOS OLIVEIRA,***.114.900-**,012****,1,APOSENTADORIA VOLUNTARIA,31/07/2018,AUDITOR-FISCAL DA RECEITA FEDERAL BRASIL,-3,Inválido,40806,DEP.DE CENTRAL.SERV.DE INATIVOS E PENS.,17000,Ministério da Economia,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,28/11/2016,NaN,10/09/2018,344,25/07/1997,NaN,DECRETO,PORTARIA,2023,3,114900,114900MARCELO RAMOS OLIVEIRA,114900MARCELO RAMOS OLIVEIRA25/07/1997,1

In [140]:
df1['tipo_aposentadoria'].value_counts() 

APOSENTADORIA VOLUNTARIA          384722
APOSENTADORIA POR INVALIDEZ        28192
APOSENTADORIA COMPULSORIA           4616
APOSENTADORIA POR INCAPACIDADE      1204
OUTROS                                58
Name: tipo_aposentadoria, dtype: int64

In [141]:
418792-417557

1235

In [138]:
418792-1528/2

418028.0

In [84]:
df23[df23['data_diploma_cpf_nome']=='354883MARIA DE LOURDES SOARES DE CASTRO08/10/1981']

,ano,mes,cpf,nome,id_servidor,tipo_aposentadoria,org_lotacao,regime_juridico,total,DATA_APOSENTADORIA,DESCRICAO_CARGO,DATA_INGRESSO_CARGOFUNCAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,cpf_nome,data_diploma_cpf_nome,duplicata_data_diploma_cpf_nome
864,2023,3,354883,MARIA DE LOURDES SOARES DE CASTRO,84618406,APOSENTADORIA VOLUNTARIA,Ministério da Justiça e Segurança Pública,REGIME JURIDICO UNICO,1,27/01/2020,MEDICO,01/07/2004,08/10/1981,354883MARIA DE LOURDES SOARES DE CASTRO,354883MARIA DE LOURDES SOARES DE CASTRO08/10/1981,True
865,2023,3,354883,MARIA DE LOURDES SOARES DE CASTRO,84618406,APOSENTADORIA VOLUNTARIA,Ministério da Justiça e Segurança Pública,REGIME JURIDICO UNICO,1,27/01/2020,MEDICO,01/07/2004,08/10/1981,354883MARIA DE LOURDES SOARES DE CASTRO,354883MARIA DE LOURDES SOARES DE CASTRO08/10/1981,True


In [ ]:
df23_semnan = df23['DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO']